[View in Colaboratory](https://colab.research.google.com/github/opooladz/ESPRIT-Autoencoder-Anomaly-Detection/blob/master/Copy_of_ESPRIT2.ipynb)

In [0]:
!apt-get install nvidia-cuda-toolkit
!pip3 install numba

import os
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/lib/nvidia-cuda-toolkit/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/lib/x86_64-linux-gnu/libnvvm.so"

from numba import cuda
import numpy as np
import time

@cuda.jit
def hello(data):
    data[cuda.blockIdx.x, cuda.threadIdx.x] = cuda.blockIdx.x

numBlocks = 5
threadsPerBlock = 10

data = np.ones((numBlocks, threadsPerBlock), dtype=np.uint8)

hello[numBlocks, threadsPerBlock](data)

print(data)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  accountsservice acpid activity-log-manager adwaita-icon-theme apg aspell
  aspell-en at-spi2-core avahi-daemon avahi-utils bbswitch-dkms bind9-host
  binfmt-support bluez bluez-obexd bsdmainutils ca-certificates-java
  cheese-common clang-3.8 cracklib-runtime crda cups-pk-helper dbus
  dbus-user-session dbus-x11 dconf-cli dconf-gsettings-backend dconf-service
  default-jre default-jre-headless desktop-file-utils dictionaries-common
  distro-info-data dkms dmsetup dns-root-data dnsmasq-base dosfstools eject
  emacsen-common enchant evolution-data-server evolution-data-server-common
  fakeroot file fontconfig fonts-dejavu-extra fuse gcr gdisk geoclue
  geoclue-ubuntu-geoip geoip-database gettext-base gir1.2-atk-1.0
  gir1.2-freedesktop gir1.2-gdkpixbuf-2.0 gir1.2-glib-2.0 gir1.2-gtk-3.0
  gir1.2-ibus-1.0 gir1.2-notify-0.7 gir1.2-packagekit

In [0]:
!pip install numba

    100% |████████████████████████████████| 1.9MB 7.9MB/s 
    100% |████████████████████████████████| 15.9MB 2.5MB/s 


In [1]:
import numpy as np
from scipy.linalg import toeplitz
from numpy import linalg as lg
from time import time
from typing import Tuple
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from numba import jit

flatten = lambda l: [item for sublist in l for item in sublist]

@jit
def compute_autocovariance(x: np.ndarray, M: int) -> np.ndarray:
    """
    This function compute the auto-covariance matrix of a numpy signal.
    The auto-covariance is computed as follows
    .. math:: \textbf{R}=\frac{1}{N}\sum_{M-1}^{N-1}\textbf{x}_{m}\textbf{x}_{m}^{H}
    where :math:`\textbf{x}_{m}^{T}=[x[m],x[m-1],x[m-M+1]]`.
    :param x: 1-D vector of size N
    :param M:  int, optional. Size of signal block.
    :returns: NxN ndarray
    """

    # Create covariance matrix for psd estimation
    # length of the vector x
    x = np.asarray(x).squeeze()
    assert x.ndim == 1, '1-D only'
    N = x.size

    # Create column vector (Nx1) from row array
    x_vect = x[None, :].T

    # init covariance matrix
    yn = x_vect[M-1::-1]  # reverse order from M-1 to 0

    R = yn @ yn.conj().T  # zeroth lag
    # about 5-8% of computation time
    for i in range(1, N-M):  # no zero because we just computed it
        # extract the column vector
        yn = x_vect[M-1+i:i-1:-1]

        R = R + yn @ yn.conj().T

    return R / N
@jit
def wrapper(x):
  return np.convolve(x,xc,mode="valid")

@jit
def esprit1(x: np.ndarray, L: int, M: int=None, fs: int=1,
           verbose: bool=False) -> Tuple[np.ndarray, np.ndarray]:
    """
    This function estimate the frequency components based on the ESPRIT algorithm [ROY89]_
    The frequencies are related to the roots as :math:`z=e^{-2j\pi f/Fe}`.
    See [STO97]_ section 4.7 for more information about the implementation.
    :param x: ndarray, Nsamples
    :param L: int. Number of components to be extracted.
    :param M:  int, optional. Size of signal block.
    :param Fs: float. Sampling Frequency.
    :returns: ndarray ndarray containing the L frequencies
    >>> import numpy as np
    >>> import spectral_analysis.spectral_analysis as sa
    >>> Fe=500
    >>> t=1.*np.arange(100)/Fe
    >>> x=np.exp(2j*np.pi*55.2*t)
    >>> f=sa.Esprit(x,1,None,Fe)
    >>> print(f)
    """

    x = np.asarray(x).squeeze()
    assert x.ndim in (1, 2)
    # length of the vector x
    if x.ndim == 1:
        N = x.size
    else:
        N = x.shape[1]

    if M is None:
        M = N // 2
# %% extract signal subspace  99.9 % of computation time
    tic = time()
    if x.ndim == 1 and isinstance(M, int):
        R = compute_autocovariance(x, M)  # 75% of computation time        
    else:
        # the random phase of transmit/receive/target actually helps--need at least 5-6 observations to make useful
        R = np.cov(x, rowvar=False)
    if verbose:
        print('autocov sec.', time()-tic)
    # R = subspace.corrmtx(x.astype(complex128),M).astype(float) #f2py fortran

    tic = time()
    #U, S, V = lg.svd(R)  # 25% of computation time
    w, v = lg.eig(R)    
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]    
    if verbose:
        print('svd sec.', time()-tic)
# %% take eigenvalues and determine sinusoid frequencies
    # Remove last row
    S1 = v[:-1, :L]
    # Remove first row
    S2 = v[1:, :L]

    # Compute matrix Phi (Stoica 4.7.12)  <0.1 % of computation time
    Phi = lg.inv(S1.conj().T @ S1) @ S1.conj().T @ S2

    # Perform eigenvalue decomposition <0.1 % of computation time
    V, U = lg.eig(Phi)

    # extract frequencies ((note that there a minus sign since Yn are defined as [y(n), y(n-1),y(n-2),..].T))
    ang = -np.angle(V)

    # frequency normalisation
    f = fs*ang / (2.*np.pi)
    t = np.arange(0, 0.01, 1/fs)
    
    x2 = np.exp(-1j*2*np.pi*t[::-1])[:,np.newaxis]**f
    #display(xc.shape)
    #ampCisoid = np.apply_along_axis(lambda q: np.convolve(q,xc,mode="valid"),axis = 0,arr=x2)
    ampCisoid = np.apply_along_axis(wrapper,axis = 0,arr=x2)
    return f, np.abs(w[:L]), np.abs(ampCisoid)*1./480., np.angle(ampCisoid)
  

def esprit2(x: np.ndarray, L: int, M: int=None, fs: int=1,
           verbose: bool=False) -> Tuple[np.ndarray, np.ndarray]:
    """
    This function estimate the frequency components based on the ESPRIT algorithm [ROY89]_
    The frequencies are related to the roots as :math:`z=e^{-2j\pi f/Fe}`.
    See [STO97]_ section 4.7 for more information about the implementation.
    :param x: ndarray, Nsamples
    :param L: int. Number of components to be extracted.
    :param M:  int, optional. Size of signal block.
    :param Fs: float. Sampling Frequency.
    :returns: ndarray ndarray containing the L frequencies
    >>> import numpy as np
    >>> import spectral_analysis.spectral_analysis as sa
    >>> Fe=500
    >>> t=1.*np.arange(100)/Fe
    >>> x=np.exp(2j*np.pi*55.2*t)
    >>> f=sa.Esprit(x,1,None,Fe)
    >>> print(f)
    """

    x = np.asarray(x).squeeze()
    assert x.ndim in (1, 2)
    # length of the vector x
    if x.ndim == 1:
        N = x.size
    else:
        N = x.shape[1]

    if M is None:
        M = N // 2
# %% extract signal subspace  99.9 % of computation time
    tic = time()
    if x.ndim == 1 and isinstance(M, int):
        R = compute_autocovariance(x, M)  # 75% of computation time
    else:
        # the random phase of transmit/receive/target actually helps--need at least 5-6 observations to make useful
        R = np.cov(x, rowvar=False)
    if verbose:
        print('autocov sec.', time()-tic)
    # R = subspace.corrmtx(x.astype(complex128),M).astype(float) #f2py fortran

    tic = time()
    U, S, V = lg.svd(R)  # 25% of computation time
    #w, v = lg.eig(R)    
    #idx = w.argsort()[::-1]   
    #w = w[idx]
    #v = v[:,idx]    
    if verbose:
        print('svd sec.', time()-tic)
# %% take eigenvalues and determine sinusoid frequencies
    # Remove last row
    S1 = U[:-1, :L]
    # Remove first row
    S2 = U[1:, :L]

    # Compute matrix Phi (Stoica 4.7.12)  <0.1 % of computation time
    Phi = lg.inv(S1.conj().T @ S1) @ S1.conj().T @ S2

    # Perform eigenvalue decomposition <0.1 % of computation time
    V, U = lg.eig(Phi)

    # extract frequencies ((note that there a minus sign since Yn are defined as [y(n), y(n-1),y(n-2),..].T))
    ang = -np.angle(V)

    # frequency normalisation
    f = fs*ang / (2.*np.pi)

    return f, S[:L]

Using TensorFlow backend.


In [0]:
from time import time
import numpy as np
from pandas import DataFrame

# Bool: Needs to be set by user 
full_anom = 1
anom_samples = 0

x = np.random.randn(4096).astype(np.complex128)
F = 12345.6

f0 = 12345.6
f1 = 1000.6
f2 = 726.6
f3 = 57.3

fs = 48e3
snr = 20.  # dB
Ntone = 6

t = np.arange(0, 0.01, 1/fs)

nvar = 10**(-snr/10.)



M = [100] * 200  # iterating over block length

py = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fortreal = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fortcmpl = DataFrame(index=M, columns=['err', 'sigma','cisoidAmp','cisoidAngle'])
fest = []
fest2 = []
cisoidAmp = []
cisoidAmp2 = []
i = 0
for m in M:
    i = i + 1
    # Generate fist tripplet of cisoid  
    # Train data start
    xc = 15.*np.exp(1j*2*np.pi*f0*t)+7.*np.exp(1j*2*np.pi*f1*t) +10.*np.exp(1j*2*np.pi*f2*t)
    xc1 = xc + np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
  
    tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc1, Ntone//2, M=m, fs=fs, verbose=False)
    fest.append(tmpFest)
    tmpAmp = tmpAmp[0]
    cisoidAmp.append(tmpAmp)
    # Train data end 
    
    # Test section start
    
    # Inject Anomaly
    if not(full_anom or i%20):
      display(i)
      for j in np.arange(anom_samples):
        display(i)
        xc2 = 15.*np.exp(1j*2*np.pi*f0*t)+7.*np.exp(1j*2*np.pi*f1*t) +10.*np.exp(1j*2*np.pi*f3*t)+ np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
        tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc2, Ntone//2, M=m, fs=fs, verbose=False)
        fest2.append(tmpFest)
        tmpAmp = tmpAmp[0]
        cisoidAmp2.append(tmpAmp) 
    # When not Anomaly use xc as baseline signal and add AWGN - Run through esprit
    xc2 = xc +  np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
    if full_anom:
      xc2 = 15.*np.exp(1j*2*np.pi*f0*t)+7.*np.exp(1j*2*np.pi*f1*t) +10.*np.exp(1j*2*np.pi*f3*t)+ np.sqrt(nvar)*(np.random.randn(t.size) + 1j*np.random.randn(t.size))
    tmpFest, sigma,tmpAmp,cisoidAngle = esprit1(xc2, Ntone//2, M=m, fs=fs, verbose=False)
    fest2.append(tmpFest)
    tmpAmp = tmpAmp[0]
    cisoidAmp2.append(tmpAmp)  
    # Test section end 

# Create train and test set     
x_train = np.hstack((cisoidAmp,fest))    
x_test = np.hstack((cisoidAmp2,fest2))
    

In [29]:
display(fest2)
display(fest)


[array([  57.3020928 ,  726.52924014, 1000.66790047]),
 array([  57.28689956,  726.60405089, 1000.64547015]),
 array([  57.28406025,  726.58086282, 1000.58935438]),
 array([  57.32119903,  726.62772058, 1000.55718363]),
 array([  57.31274635,  726.62451953, 1000.53976224]),
 array([  57.29153586,  726.59154602, 1000.56785455]),
 array([  57.27025966,  726.57731876, 1000.58135954]),
 array([  57.26040294,  726.54849429, 1000.60797735]),
 array([  57.28238329,  726.57338329, 1000.52675413]),
 array([  57.29647038,  726.52135626, 1000.53982717]),
 array([  57.31560195,  726.57518679, 1000.60158841]),
 array([  57.29966974,  726.61092588, 1000.57634088]),
 array([  57.29886033,  726.59670923, 1000.69907178]),
 array([  57.29064006,  726.59015528, 1000.49780003]),
 array([  57.32967645,  726.6216979 , 1000.64780181]),
 array([  57.29111252,  726.59516932, 1000.57253404]),
 array([  57.31114876,  726.57168141, 1000.5695246 ]),
 array([  57.27520428,  726.55084245, 1000.5814585 ]),
 array([  

[array([12345.61626302,   726.64447664,  1000.6325798 ]),
 array([12345.61942413,   726.63326453,  1000.6208261 ]),
 array([12345.58606047,   726.6434435 ,  1000.57784298]),
 array([12345.61101342,   726.60397366,  1000.61555434]),
 array([12345.61302464,   726.64018473,  1000.58605904]),
 array([12345.64275078,   726.62436449,  1000.70007624]),
 array([12345.59839237,   726.52208392,  1000.58786024]),
 array([12345.57189691,   726.56276924,  1000.58635299]),
 array([12345.6011113 ,   726.63663469,  1000.61495344]),
 array([12345.61726154,   726.55737924,  1000.6058676 ]),
 array([12345.61522918,   726.63320192,  1000.59031286]),
 array([12345.60399107,   726.59215105,  1000.63404947]),
 array([12345.61596439,   726.62647887,  1000.60280557]),
 array([12345.61199725,   726.61499544,  1000.62020925]),
 array([12345.60200572,   726.54835528,  1000.5917415 ]),
 array([12345.59991074,   726.58117038,  1000.61595154]),
 array([12345.61238821,   726.60228702,  1000.54098783]),
 array([12345.

In [28]:
from keras import regularizers
from sklearn.preprocessing import StandardScaler
# Make sure to run this after each new generation of data
# zero mean and unit var
x_train2 = StandardScaler().fit_transform(x_train)
x_test2 = StandardScaler().fit_transform(x_test)
#x_train2 = x_train2[:,:,np.newaxis]
#x_test2 = x_test2[:,:,np.newaxis]

#x_train2 = x_train2.astype('float32') / 255.
#x_test2 = x_test2.astype('float32') / 255.

display(x_train2.shape)

input_dim = x_train.shape[1]

# set size of autoencoder 
encoding_dim = 6

# use elu because it is leaky tried both net and l1 and l2 : net and l1 worked the best 
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="elu",kernel_initializer= 'he_normal', activity_regularizer=regularizers.l1_l2(l1=10e-5, l2=10e-4))(input_layer)
#encoder = Dense(encoding_dim, activation="elu",activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2), activation="elu",kernel_initializer= 'he_normal')(encoder)
decoder = Dense(int(encoding_dim / 2), activation='elu',kernel_initializer= 'he_normal')(encoder)
decoder = Dense(input_dim, activation='elu',kernel_initializer= 'he_normal')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

autoencoder.summary()  


(200, 6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 42        
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 21        
_________________________________________________________________
dense_19 (Dense)             (None, 3)                 12        
_________________________________________________________________
dense_20 (Dense)             (None, 6)                 24        
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________


In [5]:
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy_with_masking', factor=0.2,
                              patience=5,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 0s 2ms/step - loss: 0.5229 - acc: 0.6167 - val_loss: 0.6079 - val_acc: 0.7500
Epoch 2/800
180/180 [==============================] - 0s 278us/step - loss: 0.5127 - acc: 0.6278 - val_loss: 0.5990 - val_acc: 0.7500
Epoch 3/800
180/180 [==============================] - 0s 307us/step - loss: 0.5022 - acc: 0.6333 - val_loss: 0.5901 - val_acc: 0.7500
Epoch 4/800
180/180 [==============================] - 0s 290us/step - loss: 0.4927 - acc: 0.6333 - val_loss: 0.5807 - val_acc: 0.7000
Epoch 5/800
 32/180 [====>.........................] - ETA: 0s - loss: 0.4667 - acc: 0.7500

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:972: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_accuracy_with_masking` which is not available. Available metrics are: val_loss,val_acc,loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


180/180 [==============================] - 0s 320us/step - loss: 0.4829 - acc: 0.6389 - val_loss: 0.5705 - val_acc: 0.7000
Epoch 6/800
180/180 [==============================] - 0s 299us/step - loss: 0.4734 - acc: 0.6333 - val_loss: 0.5620 - val_acc: 0.6500
Epoch 7/800
180/180 [==============================] - 0s 326us/step - loss: 0.4632 - acc: 0.6556 - val_loss: 0.5519 - val_acc: 0.6500
Epoch 8/800
180/180 [==============================] - 0s 373us/step - loss: 0.4528 - acc: 0.6611 - val_loss: 0.5431 - val_acc: 0.6500
Epoch 9/800
180/180 [==============================] - 0s 320us/step - loss: 0.4432 - acc: 0.6667 - val_loss: 0.5324 - val_acc: 0.6500
Epoch 10/800
180/180 [==============================] - 0s 312us/step - loss: 0.4334 - acc: 0.6833 - val_loss: 0.5234 - val_acc: 0.6500
Epoch 11/800
180/180 [==============================] - 0s 306us/step - loss: 0.4246 - acc: 0.7000 - val_loss: 0.5132 - val_acc: 0.7000
Epoch 12/800
180/180 [==============================] - 0s 313us/

'[test_loss, test_acc]'

[0.11783276438713074, 0.945]

In [24]:
from keras import regularizers
from sklearn.preprocessing import StandardScaler
# Make sure to run this after each new generation of data
# zero mean and unit var
x_train2 = StandardScaler().fit_transform(x_train)
x_test2 = StandardScaler().fit_transform(x_test)
#x_train2 = x_train2[:,:,np.newaxis]
#x_test2 = x_test2[:,:,np.newaxis]

#x_train2 = x_train2.astype('float32') / 255.
#x_test2 = x_test2.astype('float32') / 255.

display(x_train2.shape)

input_dim = x_train.shape[1]

# set size of autoencoder 
encoding_dim = 6
elu = keras.layers.ELU(alpha=1.2)
lrlu= keras.layers.LeakyReLU(alpha=0.3)


# use elu because it is leaky tried both net and l1 and l2 : net and l1 worked the best 
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim,kernel_initializer= 'he_normal', activity_regularizer=regularizers.l1_l2(l1=10e-5, l2=75.10e-5))(input_layer)
encoder = keras.layers.LeakyReLU(alpha=0.011)(encoder)
#encoder = Dense(encoding_dim, activation="elu",activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoder = Dense(int(encoding_dim / 2),kernel_initializer= 'he_normal')(encoder)
encoder = keras.layers.LeakyReLU(alpha=0.011)(encoder)
decoder = Dense(int(encoding_dim / 2),kernel_initializer= 'he_normal')(encoder)
decoder = keras.layers.LeakyReLU(alpha=0.011)(decoder)
decoder = Dense(input_dim,kernel_initializer= 'he_normal')(decoder)
decoder = keras.layers.LeakyReLU(alpha=0.011)(decoder)
autoencoder2 = Model(inputs=input_layer, outputs=decoder)

autoencoder2.summary()  


(200, 6)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 6)                 0         
_________________________________________________________________
dense_13 (Dense)             (None, 6)                 42        
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 21        
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 3)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 12        
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 3)                 0         
__________

In [0]:
#Results with no anomily 
# Loss and Val Loss are very close + Accuracy and Val Accuracy are very close 
# Depending on the params of the net it can hit Accuracy of  1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
full_anom = 0
anom_samples = 0
batch_size = 32
import keras

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
nadam =keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

autoencoder2.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    #loss = 'binary_crossentropy',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder2.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])
score = autoencoder2.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)
display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 7s 38ms/step - loss: 0.6340 - acc: 0.5222 - val_loss: 0.6486 - val_acc: 0.4000
Epoch 2/800
180/180 [==============================] - 0s 363us/step - loss: 0.6323 - acc: 0.5167 - val_loss: 0.6465 - val_acc: 0.4000
Epoch 3/800
180/180 [==============================] - 0s 358us/step - loss: 0.6305 - acc: 0.5333 - val_loss: 0.6434 - val_acc: 0.4000
Epoch 4/800
180/180 [==============================] - 0s 359us/step - loss: 0.6292 - acc: 0.5389 - val_loss: 0.6430 - val_acc: 0.4000
Epoch 5/800
180/180 [==============================] - 0s 364us/step - loss: 0.6275 - acc: 0.5611 - val_loss: 0.6400 - val_acc: 0.4000
Epoch 6/800
180/180 [==============================] - 0s 347us/step - loss: 0.6262 - acc: 0.5778 - val_loss: 0.6392 - val_acc: 0.4500
Epoch 7/800
180/180 [==============================] - 0s 354us/step - loss: 0.6250 - acc: 0.6111 - val_loss: 0.6368 - val_acc: 0.5000
Epoch 8/800

'[test_loss, test_acc]'

[0.43619103610515597, 0.945]

In [0]:

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 8s 46ms/step - loss: 0.1360 - acc: 0.7833 - val_loss: 0.1098 - val_acc: 0.7000
Epoch 2/800
180/180 [==============================] - 0s 359us/step - loss: 0.1356 - acc: 0.7889 - val_loss: 0.1042 - val_acc: 0.7000
Epoch 3/800
180/180 [==============================] - 0s 362us/step - loss: 0.1352 - acc: 0.8000 - val_loss: 0.1043 - val_acc: 0.7000
Epoch 4/800
180/180 [==============================] - 0s 363us/step - loss: 0.1345 - acc: 0.7944 - val_loss: 0.1032 - val_acc: 0.7000
Epoch 5/800
180/180 [==============================] - 0s 358us/step - loss: 0.1344 - acc: 0.8167 - val_loss: 0.1041 - val_acc: 0.7000
Epoch 6/800
180/180 [==============================] - 0s 354us/step - loss: 0.1337 - acc: 0.8167 - val_loss: 0.1047 - val_acc: 0.7000
Epoch 7/800
180/180 [==============================] - 0s 366us/step - loss: 0.1335 - acc: 0.8222 - val_loss: 0.1051 - val_acc: 0.7000
Epoch 8/800

'[test_loss, test_acc]'

[37.95506736755371, 0.12]

In [25]:

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 0s 2ms/step - loss: 1.0297 - acc: 0.1500 - val_loss: 1.0865 - val_acc: 0.1500
Epoch 2/800
180/180 [==============================] - 0s 253us/step - loss: 0.9824 - acc: 0.1778 - val_loss: 1.0376 - val_acc: 0.1500
Epoch 3/800
180/180 [==============================] - 0s 246us/step - loss: 0.9403 - acc: 0.2056 - val_loss: 0.9952 - val_acc: 0.2000
Epoch 4/800
180/180 [==============================] - 0s 263us/step - loss: 0.8996 - acc: 0.2556 - val_loss: 0.9593 - val_acc: 0.2500
Epoch 5/800
180/180 [==============================] - 0s 245us/step - loss: 0.8636 - acc: 0.3056 - val_loss: 0.9294 - val_acc: 0.3500
Epoch 6/800
180/180 [==============================] - 0s 257us/step - loss: 0.8323 - acc: 0.3778 - val_loss: 0.9042 - val_acc: 0.4000
Epoch 7/800
180/180 [==============================] - 0s 261us/step - loss: 0.8053 - acc: 0.3944 - val_loss: 0.8826 - val_acc: 0.3500
Epoch 8/800


'[test_loss, test_acc]'

[0.10375401735305786, 0.795]

In [26]:

#Results with full anomily 
full_anom = 1 
anom_samples = 0
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.predict( x_test2, batch_size=32, verbose=1, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 0s 2ms/step - loss: 0.1210 - acc: 0.8889 - val_loss: 0.1780 - val_acc: 0.8000
Epoch 2/800
180/180 [==============================] - 0s 266us/step - loss: 0.1199 - acc: 0.8778 - val_loss: 0.1782 - val_acc: 0.8000
Epoch 3/800
180/180 [==============================] - 0s 266us/step - loss: 0.1195 - acc: 0.8889 - val_loss: 0.1755 - val_acc: 0.8500
Epoch 4/800
180/180 [==============================] - 0s 247us/step - loss: 0.1200 - acc: 0.8833 - val_loss: 0.1730 - val_acc: 0.9000
Epoch 5/800
180/180 [==============================] - 0s 257us/step - loss: 0.1195 - acc: 0.8722 - val_loss: 0.1748 - val_acc: 1.0000
Epoch 6/800
180/180 [==============================] - 0s 262us/step - loss: 0.1197 - acc: 0.8778 - val_loss: 0.1732 - val_acc: 0.8500
Epoch 7/800
180/180 [==============================] - 0s 251us/step - loss: 0.1196 - acc: 0.8778 - val_loss: 0.1735 - val_acc: 0.8500
Epoch 8/800


'[test_loss, test_acc]'

array([[-0.05633086,  1.9496233 ,  1.363966  , -0.1084106 , -0.89158803,
         1.3663949 ],
       [-0.7465948 , -0.225851  ,  1.1228017 , -0.78701407,  0.05606425,
         1.124202  ],
       [-0.8247652 ,  0.47963095, -0.3069414 , -0.8577421 , -0.52891135,
        -0.30963695],
       ...,
       [ 1.7349085 , -0.6535325 ,  0.40537345,  1.8307478 ,  0.74874   ,
         0.40446252],
       [-0.58899045, -0.20738149,  1.4146882 , -0.63941365,  0.03107619,
         1.4171655 ],
       [ 0.3978861 ,  0.07284081, -0.97770995,  0.3691734 , -0.43054497,
        -0.9780622 ]], dtype=float32)

In [30]:
display(score[0,:])
display(x_test2[0,:])
display(x_train2[0,:])
display(x_train[0,:])

array([-0.05633086,  1.9496233 ,  1.363966  , -0.1084106 , -0.89158803,
        1.3663949 ], dtype=float32)

array([ 0.02897468,  1.94101657,  1.45149563,  0.02839605, -1.95343728,
        1.45383821])

array([ 0.93952285, -1.37852119,  0.6397283 ,  0.95364096,  1.37335243,
        0.6383703 ])

array([1.50334613e+01, 9.63636029e+00, 6.49541661e+00, 1.23456163e+04,
       7.26644477e+02, 1.00063258e+03])

In [0]:
#Results with one sample anomily 
# the duration of the anom can be tuned with anom_samples
full_anom = 0
anom_samples = 1
nb_epoch = 800 # a bit overkill but wanted to see how it progressed w more epochs 
batch_size = 32
import keras
autoencoder.compile(optimizer='nadam', 
                    loss='mean_squared_error', 
                    #loss='kullback_leibler_divergence',
                    metrics=['accuracy'])
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_acc', factor=0.2,
                              patience=300,verbose = 1)
history = autoencoder.fit(x_train2, x_train2,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    #validation_data=(x_test2[0:200,:], x_test2[0:200,:]),
                    validation_split=.1,
                    verbose=1,callbacks=[reduce_lr])

score = autoencoder.evaluate(x_test2, x_test2, batch_size=32, verbose=1, sample_weight=None, steps=None)

display('[test_loss, test_acc]')
display(score)

Train on 180 samples, validate on 20 samples
Epoch 1/800
180/180 [==============================] - 9s 48ms/step - loss: 1.5007 - acc: 0.1944 - val_loss: 1.4750 - val_acc: 0.0500
Epoch 2/800
180/180 [==============================] - 0s 353us/step - loss: 1.4188 - acc: 0.1778 - val_loss: 1.3748 - val_acc: 0.1000
Epoch 3/800
180/180 [==============================] - 0s 345us/step - loss: 1.3436 - acc: 0.1778 - val_loss: 1.2763 - val_acc: 0.1000
Epoch 4/800
180/180 [==============================] - 0s 350us/step - loss: 1.2729 - acc: 0.1889 - val_loss: 1.1892 - val_acc: 0.1000
Epoch 5/800
180/180 [==============================] - 0s 372us/step - loss: 1.2061 - acc: 0.2000 - val_loss: 1.1073 - val_acc: 0.0500
Epoch 6/800
180/180 [==============================] - 0s 348us/step - loss: 1.1525 - acc: 0.2222 - val_loss: 1.0312 - val_acc: 0.0500
Epoch 7/800
180/180 [==============================] - 0s 366us/step - loss: 1.0978 - acc: 0.2389 - val_loss: 0.9676 - val_acc: 0.0500
Epoch 8/800

'[test_loss, test_acc]'

[5.602794224875314, 0.276190476332392]